In [1]:
import tensorflow as tf
sess = tf.Session()

In [2]:
a = tf.constant([1.0,2.0], name = "a")
b = tf.constant([2.0,3.0], name = "b")
result = a + b

In [3]:
sess.run(b)

array([2., 3.], dtype=float32)

变量声明变量tf.Variable<br/>
生成随机矩阵2x3 tf.random_normal<br/>
标准差stddev=2<br/>
均值mean = 0 (default)

In [4]:
weights = tf.Variable(tf.random_normal([2,3],stddev = 2))

In [5]:
bias = tf.Variable(tf.zeros(1,1))

In [6]:
#w1,w2两个变量
w1 = tf.Variable(tf.random_normal((2,3),seed=1,stddev=1))
w2 = tf.Variable(tf.random_normal((3,1),seed=1,stddev=1))

#定义x为常量
x = tf.constant([[.7,0.9]])

#前向传播
a = tf.matmul(x, w1)
y = tf.matmul(w1, w2)

sess = tf.Session()

sess.run(w1.initializer)
sess.run(w2.initializer)
print(sess.run(y))
sess.close()

[[2.8665385]
 [2.1677787]]


In [7]:
w1 = tf.Variable(tf.random_normal((2,3),seed = 1,stddev = 1))
w2 = tf.Variable(tf.random_normal((3,1),seed = 1,stddev = 1))

In [8]:
x = tf.placeholder(tf.float32, shape = (1, 2), name = "input")
a = tf.matmul(x, w1)
y = tf.matmul(a, w2)

In [9]:
init_op = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init_op)

In [12]:
#sess.run(y)
sess.run(y,feed_dict={x:[[0.7, 0.9]]})

array([[3.957578]], dtype=float32)

In [19]:
y = tf.placeholder(tf.float32, shape=(1,1),name="y-input")
y = tf.sigmoid(y)

In [20]:
s = tf.clip_by_value(y,1e-10,1.0)

In [36]:
#loss
y_ = tf.placeholder(tf.float32, shape=(None,1), name = "y-input")
cross_entropy = \
    -tf.reduce_mean(y_ * tf.log(tf.clip_by_value(y,1e-10,1.0))) + \
    (1-y) * tf.log(tf.clip_by_value(1-y,1e-10,1.0))

In [37]:
learning_rate = 0.001

In [35]:
#train_step = \tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy)

In [ ]:
# 定义神经网络的相关参数和变量
batch_size = 8
x = tf.placeholder(tf.float32, shape=(None, 2), name="x-input")
y_ = tf.placeholder(tf.float32, shape=(None, 1), name='y-input')
w1= tf.Variable(tf.random_normal([2, 1], stddev=1, seed=1))
y = tf.matmul(x, w1)

# 设置自定义的损失函数
# 定义损失函数使得预测少了的损失大，于是模型应该偏向多的方向预测。
loss_less = 10
loss_more = 1
loss = tf.reduce_sum(tf.where(tf.greater(y, y_), (y - y_) * loss_more, (y_ - y) * loss_less))
train_step = tf.train.AdamOptimizer(0.001).minimize(loss)

# 生成模拟数据集
rdm = RandomState(1)
X = rdm.rand(128,2)
Y = [[x1+x2+(rdm.rand()/10.0-0.05)] for (x1, x2) in X]

# 训练模型
with tf.Session() as sess:
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    STEPS = 5000
    for i in range(STEPS):
        start = (i*batch_size) % 128
        end = (i*batch_size) % 128 + batch_size
        sess.run(train_step, feed_dict={x: X[start:end], y_: Y[start:end]})
        if i % 1000 == 0:
            print("After %d training step(s), w1 is: " % (i))
            print(sess.run(w1), "\n")
    print("Final w1 is: \n", sess.run(w1))

In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import os

INPUT_NODE = 784
OUTPUT_NODE = 10
LAYER1_NODE = 500
BATCH_SIZE = 100
LEARNING_RATE_BASE = 0.8
LEARNING_RATE_DECAY = 0.99
REGULARIZATION_RATE = 0.0001
TRAINING_STEPS = 30000
MOVING_AVERAGE_DECAY = 0.99
MODEL_SAVE_PATH="MNIST_model/"
MODEL_NAME="mnist_model"

def get_weight_variable(shape, regularizer):
    weights = tf.get_variable("weights", shape, initializer=tf.truncated_normal_initializer(stddev=0.1))
    if regularizer != None: tf.add_to_collection('losses', regularizer(weights))
    return weights


def inference(input_tensor, regularizer):
    with tf.variable_scope('layer1'):

        weights = get_weight_variable([INPUT_NODE, LAYER1_NODE], regularizer)
        biases = tf.get_variable("biases", [LAYER1_NODE], initializer=tf.constant_initializer(0.0))
        layer1 = tf.nn.relu(tf.matmul(input_tensor, weights) + biases)

    with tf.variable_scope('layer2'):
        weights = get_weight_variable([LAYER1_NODE, OUTPUT_NODE], regularizer)
        biases = tf.get_variable("biases", [OUTPUT_NODE], initializer=tf.constant_initializer(0.0))
        layer2 = tf.matmul(layer1, weights) + biases

    return layer2

def train(mnist):

    x = tf.placeholder(tf.float32, [None, INPUT_NODE], name='x-input')
    y_ = tf.placeholder(tf.float32, [None, OUTPUT_NODE], name='y-input')

    regularizer = tf.contrib.layers.l2_regularizer(REGULARIZATION_RATE)
    y = inference(x, regularizer)
    global_step = tf.Variable(0, trainable=False)


    variable_averages = tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY, global_step)
    variables_averages_op = variable_averages.apply(tf.trainable_variables())
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(y, tf.argmax(y_, 1))
    cross_entropy_mean = tf.reduce_mean(cross_entropy)
    loss = cross_entropy_mean + tf.add_n(tf.get_collection('losses'))
    learning_rate = tf.train.exponential_decay(
        LEARNING_RATE_BASE,
        global_step,
        mnist.train.num_examples / BATCH_SIZE, LEARNING_RATE_DECAY,
        staircase=True)
    train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    with tf.control_dependencies([train_step, variables_averages_op]):
        train_op = tf.no_op(name='train')


    saver = tf.train.Saver()
    with tf.Session() as sess:
        tf.global_variables_initializer().run()

        for i in range(TRAINING_STEPS):
            xs, ys = mnist.train.next_batch(BATCH_SIZE)
            _, loss_value, step = sess.run([train_op, loss, global_step], feed_dict={x: xs, y_: ys})
            if i % 1000 == 0:
                print("After %d training step(s), loss on training batch is %g." % (step, loss_value))
                saver.save(sess, os.path.join(MODEL_SAVE_PATH, MODEL_NAME), global_step=global_step)


def main(argv=None):
    mnist = input_data.read_data_sets("../../datasets/MNIST_data/", one_hot=True)
    train(mnist)

if __name__ == '__main__':
    tf.app.run()

C:\Users\adward\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Extracting ../../datasets/MNIST_data/train-images-idx3-ubyte.gz


Extracting ../../datasets/MNIST_data/train-labels-idx1-ubyte.gz
Extracting ../../datasets/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ../../datasets/MNIST_data/t10k-labels-idx1-ubyte.gz


ValueError: Only call `sparse_softmax_cross_entropy_with_logits` with named arguments (labels=..., logits=..., ...)